# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with:
1. cd "C:\Users\e.a.wright\nosql-challenge\Starter_Code\Resources"
2. mongoimport --db uk_food --collection establishments --file establishments.json --jsonArray

In [134]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [135]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [136]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'fruits_db', 'gardenDB', 'local', 'met', 'petsitly_marketing', 'travel_db', 'uk_food']


In [137]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [138]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [139]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6499b61206aa30e49cba4d16'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [140]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [141]:
# Create a dictionary for the new restaurant data
document = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [142]:
# Insert the new restaurant into the collection
result = establishments.insert_one(document)

In [143]:
# Check that the new restaurant was inserted
if result.inserted_id:
    print("Document inserted successfully.")
else:
    print("Failed to insert document.")

Document inserted successfully.


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [144]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type_id = establishments.find_one({"BusinessType":"Restaurant/Cafe/Canteen"}, {"BusinessTypeID":1, "BusinessType":1, "_id":0})

In [145]:
business_type_id

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}

3. Update the new restaurant with the `BusinessTypeID` you found.

In [146]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one({"BusinessName": "Penang Flavours"}, {"$set":{"BusinessTypeID":business_type_id["BusinessTypeID"]}})

In [147]:
# Confirm that the new restaurant was updated
establishments.find_one({"BusinessName":"Penang Flavours"}, {"BusinessName":1,"BusinessTypeID":1, "BusinessType":1, "_id":0})

{'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1}

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [148]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName":"Dover"})
print(f"There are {dover_count} establishments in Dover.")

There are 994 establishments in Dover.


In [149]:
# Delete all documents where LocalAuthorityName is "Dover"
deleteDover = establishments.delete_many({"LocalAuthorityName":"Dover"})
# Confirm that the documents were deleted
deleteDover.deleted_count

994

In [150]:
# Check if any remaining documents include Dover
filter = {
    "LocalAuthorityName": "Dover"
}

# Retrieve a count of the documents abiding by the filter
count = establishments.count_documents(filter)

# Print the remaining document count in an f string
print(f'There are {count} documents remaining with the LocalAuthorityName "Dover"')

There are 0 documents remaining with the LocalAuthorityName "Dover"


In [151]:
# Check that other documents remain with 'find_one'
establishments.find_one()

{'_id': ObjectId('6499b61206aa30e49cba4ffd'),
 'FHRSID': 647177,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT19 6PY',
 'Phone': '',
 'RatingValue': '4',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingDate': '2014-03-31T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 10},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.196408', 'latitude': '51.086058'},
 'RightToReply': '',
 'Distance': 4591.821311183521,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  '

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [152]:
# Convert geocode.longitude and geocode.latitude to doubles
conversion_operation = [
    {
        '$set': {
            'geocode.longitude': {
                '$convert': {
                    'input': '$geocode.longitude',
                    'to': 'double',
                    'onError': 0.0
                }
            },
            'geocode.latitude': {
                '$convert': {
                    'input': '$geocode.latitude',
                    'to': 'double',
                    'onError': 0.0
                }
            }
        }
    }
]
# Update the documents using the conversion_operation
result = establishments.update_many({}, conversion_operation)

# Confirm the update by printing the first 5 documents
pprint(list(establishments.find().limit(5)))

[{'AddressLine1': 'Wear Bay Road',
  'AddressLine2': 'Folkestone',
  'AddressLine3': 'Kent',
  'AddressLine4': '',
  'BusinessName': 'Wear Bay Bowls Club',
  'BusinessType': 'Pub/bar/nightclub',
  'BusinessTypeID': 7843,
  'ChangesByServerID': 0,
  'Distance': 4591.821311183521,
  'FHRSID': 647177,
  'LocalAuthorityBusinessID': 'PI/000041489',
  'LocalAuthorityCode': '188',
  'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
  'LocalAuthorityName': 'Folkestone and Hythe',
  'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
  'NewRatingPending': False,
  'Phone': '',
  'PostCode': 'CT19 6PY',
  'RatingDate': '2014-03-31T00:00:00',
  'RatingKey': 'fhrs_4_en-gb',
  'RatingValue': '4',
  'RightToReply': '',
  'SchemeType': 'FHRS',
  '_id': ObjectId('6499b61206aa30e49cba4ffd'),
  'geocode': {'latitude': 51.086058, 'longitude': 1.196408},
  'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
             'rel': 'self'}],
  'meta': {'dataSo

Use `update_many` to convert `RatingValue` to integer numbers.

In [153]:
# Get the unique values in the RatingValue field from establishments
ratings = establishments.distinct("RatingValue")
# Print the unique values
print(ratings)

['0', '1', '2', '3', '4', '5', 'Awaiting Inspection', 'AwaitingInspection', 'AwaitingPublication', 'Exempt', 'Pass']


In [154]:
# Coerce the non-numeric values (including 0 because it is outside of the 1-5 range) in RatingValue to null

# Create a list of numeric strings of the RatingValue field (1-5)
numeric_ratings = ['1','2','3','4','5']
# Set non 1-5 Rating Values to Null
establishments.update_many({"RatingValue":{"$nin":numeric_ratings}}, {"$set":{"RatingValue":None}})

# Query the database for documents with RatingValue of Null
null_ratings = establishments.count_documents({"RatingValue":None})
# Print the count of the documents with RatingValue of Null
print(f'There are now {null_ratings} documents with a null RatingValue.')

There are now 4157 documents with a null RatingValue.


In [158]:
# Change the data type from String to Integer for RatingValue using update_many()
conversion_operation = [
    {
        '$set': {
            'RatingValue' : {
                '$convert' : {
                    'input' : '$RatingValue',
                    'to' : 'int',
                    'onError' : 0
                }
            }
        }
    }
]
# Update the documents using the conversion_operation
result = establishments.update_many({}, conversion_operation)

In [160]:
# Confirm that RatingValue is now an integer by printing the first 5 documents
pprint(list(establishments.find().limit(5)))

[{'AddressLine1': 'Wear Bay Road',
  'AddressLine2': 'Folkestone',
  'AddressLine3': 'Kent',
  'AddressLine4': '',
  'BusinessName': 'Wear Bay Bowls Club',
  'BusinessType': 'Pub/bar/nightclub',
  'BusinessTypeID': 7843,
  'ChangesByServerID': 0,
  'Distance': 4591.821311183521,
  'FHRSID': 647177,
  'LocalAuthorityBusinessID': 'PI/000041489',
  'LocalAuthorityCode': '188',
  'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
  'LocalAuthorityName': 'Folkestone and Hythe',
  'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
  'NewRatingPending': False,
  'Phone': '',
  'PostCode': 'CT19 6PY',
  'RatingDate': '2014-03-31T00:00:00',
  'RatingKey': 'fhrs_4_en-gb',
  'RatingValue': 4,
  'RightToReply': '',
  'SchemeType': 'FHRS',
  '_id': ObjectId('6499b61206aa30e49cba4ffd'),
  'geocode': {'latitude': 51.086058, 'longitude': 1.196408},
  'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
             'rel': 'self'}],
  'meta': {'dataSour

In [161]:
# Close the connection to the database
mongo.close()

### The following code converts RatingValue dtype using a for loop instead of update_many()

In [156]:
# # Change the data type from String to Integer for RatingValue
# documents = establishments.find({"RatingValue": {"$exists": True}})
# for doc in documents:
#     rating_value = doc["RatingValue"]
#     if rating_value is not None:
#         try:
#             new_rating_value = int(rating_value)
#             establishments.update_one({"_id": doc["_id"]}, {"$set": {"RatingValue": new_rating_value}})
#             print(f'Updated {doc["_id"]} to {new_rating_value}.')
#         except ValueError:
#             print(f'Could not convert {rating_value} to an integer.')
#     else:
#         print('RatingValue is None')